In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 3.9MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ P

In [ ]:
#!/usr/bin/env python

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime as dt
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options =chrome_options)
SOURCE = "https://dashboard.madiunkota.go.id/"

In [ ]:
wd.get(SOURCE)
wd.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[2]/div[1]/div[9]/div/div[2]/div/div/div[1]/div[2]/label/select/option[4]").click()
content = wd.page_source
soup = BeautifulSoup(content, features="html.parser")
table = soup.find("table", "table table-bordered table-hover datatable-basic dataTable no-footer")
td = (table.find_all('td'))


In [ ]:
def ChangeDatetoRightFormat(string_date):
  kamus = ['januari', 'februari', 'maret', 'april', 'mei', 'juni','juli','agustus', 'september', 'oktober','november','desember']
  date_hari, date_bulan, date_tahun = string_date[:2],string_date[3:-5],string_date[-4:]
  date_bulan = kamus.index(date_bulan.lower())+1
  if(int(date_hari)<10):
    date_hari= "0"+date_hari
  if(date_bulan<10):
    date_bulan="0{}".format(date_bulan)
  return "{}-{}-{}".format(date_tahun,date_bulan,date_hari)

In [ ]:
def tabulasi_data(ldict,td):
  td = [x.get_text().strip().capitalize() if x.get_text() !=" " else "0" for x in td]
  ldict["kelurahan"].append(td[1])
  ldict["total_positif"].append(int(td[6]))
  ldict["positif_sembuh"].append(int(td[7]))
  ldict["positif_dirawat"].append(int(td[8]))
  ldict["positif_isolasi"].append(int(td[9]))
  ldict["positif_meninggal"].append(int(td[10]))

In [ ]:
import re
date = soup.find("div",class_="col-sm-12 col-md-12")
date = date.find_all("span")
date[1].get_text()
string_date = re.search(r":(.[\w ]*),",date[1].get_text()).group(1).strip()
formatted_date = ChangeDatetoRightFormat(string_date)
scrape_date = dt.now().strftime('%Y-%m-%d')

In [ ]:
keys = ["kelurahan","total_positif","positif_sembuh","positif_dirawat","positif_isolasi","positif_meninggal"]
Ldict = dict(zip(keys, ([] for _ in keys)))

In [ ]:
for i in range(0,len(td),12):
  tabulasi_data(Ldict,td[i:i+11])

In [ ]:
df = pd.DataFrame()
lenghtDF = len(Ldict["kelurahan"])
df['scrape_date'] = [scrape_date]*lenghtDF
df['date_update'] = [formatted_date]*lenghtDF
df['provinsi'] = ["Jawa Timur"]*lenghtDF
df['kabkot'] = ["Madiun"]*lenghtDF
df['Kecamatan'] = ["Null"]*lenghtDF
df["Kelurahan"] = Ldict["kelurahan"]
df['total_odp'] = 0*lenghtDF
df['total_pdp'] = 0**lenghtDF
df['positif_sembuh'] = Ldict["positif_sembuh"]
df['positif_dirawat'] = Ldict["positif_dirawat"]
df['positif_isolasi'] = Ldict["positif_isolasi"]
df['positif_meninggal'] = Ldict["positif_meninggal"]
df['source_link'] = [SOURCE]*lenghtDF
df['user_pic'] = ["Zaidan Naufal Sudrajat"]*lenghtDF
df

,scrape_date,date_update,provinsi,kabkot,Kecamatan,Kelurahan,total_odp,total_pdp,positif_sembuh,positif_dirawat,positif_isolasi,positif_meninggal,source_link,user_pic
0,2021-07-11,2021-07-11,Jawa Timur,Madiun,Null,Banjarejo,0,0,152,5,17,8,https://dashboard.madiunkota.go.id/,Zaidan Naufal Sudrajat
1,2021-07-11,2021-07-11,Jawa Timur,Madiun,Null,Demangan,0,0,146,0,11,7,https://dashboard.madiunkota.go.id/,Zaidan Naufal Sudrajat
2,2021-07-11,2021-07-11,Jawa Timur,Madiun,Null,Josenan,0,0,92,3,17,8,https://dashboard.madiunkota.go.id/,Zaidan Naufal Sudrajat
3,2021-07-11,2021-07-11,Jawa Timur,Madiun,Null,Kanigoro,0,0,159,1,40,13,https://dashboard.madiunkota.go.id/,Zaidan Naufal Sudrajat
4,2021-07-11,2021-07-11,Jawa Timur,Madiun,Null,Kartoharjo,0,0,61,2,1,5,https://dashboard.madiunkota.go.id/,Zaidan Naufal Sudrajat
5,2021-07-11,2021-07-11,Jawa Timur,Madiun,Null,Kejuron,0,0,158,1,9,17,https://dashboard.madiunkota.go.id/,Zaidan Naufal Sudrajat
6,2021-07-11,2021-07-11,Jawa Timur,Madiun,Null,Kelun,0,0,86,0,12,6,https://dashboard.madiunkota.go.id/,Zaidan Naufal Sudrajat
7,2021-07-11,2021-07-11,Jawa Timur,Madiun,Null,Klegen,0,0,158,4,49,19,https://dashboard.madiunkota.go.id/,Zaidan Naufal Sudrajat
8,2021-07-11,2021-07-11,Jawa Timur,Madiun,Null,Kuncen,0,0,6,0,3,2,https://dashboard.madiunkota.go.id/,Zaidan Naufal Sudrajat
9,2021-07-11,2021-07-11,Jawa Timur,Madiun,Null,Madiun lor,0,0,134,4,19,18,https://dashboard.madiunkota.go.id/,Zaidan Naufal Sudrajat
